# SSC Data Science and Analytics Workshop 2022

### Intro to Databases in Industry: Data Cleaning, Querying, and Modeling at Scale
---------------

SQL is powerful, fast, and reliable. But unfortunately, queries can quickly become complex, even for routine data wrangling. 

Languages like R  and Python have powerful packages, such as `tidyverse` and `pandas`, that are designed to facilitate wrangling and cleaning data. The disadvantage is that they are not as fast as SQL. Luckily for us, we can connect R directly to the databases. Not only that, but we can use the usual `tidyverse` verbs, and `dbplyr` will generate the SQL queries for us! So we can have the best of both worlds! 

In this part of the workshop, we will explore the R$\leftrightarrow$SQL interface. 




## 1. Connecting `R` to a database 

We will use the `DBI` package to connect R to a database. There are many different database management systems (DBMS) vendors out there (e.g., Oracle, Microsoft, Postgres, MySQL). Although all these DBMS are somewhat similar, they have some differences. For this reason, we need to tell the `DBI` package which database we want to connect to. In our case here, we are using the `PostgreSQL` DBMS. We need to install the PostgreSQL backend for DBI, which is the package `RPostgres`. 

Finally, the package [dbplyr](https://dbplyr.tidyverse.org/) creates the interface with the database and converts the `dplyr` verbs into SQL queries. How does that work? Very similarly to if you had loaded the tables into R as data frames.


Let's start by creating the connection.

In [1]:
library(tidyverse) # dbplyr is part of tidyverse metapackage
library(RPostgres)

-- Attaching packages ------------------------------------------------------------------------------- tidyverse 1.3.1 --

v ggplot2 3.3.5     v purrr   0.3.4
v tibble  3.1.6     v dplyr   1.0.7
v tidyr   1.1.4     v stringr 1.4.0
v readr   2.1.1     v forcats 0.5.1

-- Conflicts ---------------------------------------------------------------------------------- tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()

Warning message:
"package 'RPostgres' was built under R version 4.1.3"


**Exercise 1.1** 

Connect R to the imdb database located at [...]. 

In [13]:
# connection = dbConnect(
#     drv = ..., 
#     user = ..., 
#     password = ..., 
#     port = 5432, # this is the default port for postgres 
#     dbname = ..., 
#     host = ...)

### BEGIN SOLUTION
connection = dbConnect(
    drv = Postgres(), 
    user = "postgres", 
    password = "1645", 
    port = 5432, # this is the default port for postgres 
    dbname = "imdb", 
    host = "localhost")
### END SOLUTION

Congratulations!!! R is now connected to the database. 

## 2. Retrieving data from a database with R

Now that we have the connection ready to go, we can pull data from the database. But before we start pulling data from tables, it is useful to get some information about the database itself (e.g., what tables there are in a database, what are the fields of a table): 

- `dbListTables(connection)`: list all tables in the database accessed in connection;+
- `dbListFields(connection, table_name)`: List all columns of `table_name` in the database 

**Exercise 2.1**

List all tables of the `imdb` database. 


In [14]:
# Your code goes here. 

### BEGIN SOLUTION
dbListTables(connection)
### END SOLUTION

[1] "principals"   "movies"       "acting_roles" "names"        "aliases"     
[6] "crew_roles"   "movie_genres"

**Exercise 2.2**

List all columns from of the `movies` relation in the `imdb` database. (Note: relation is just another name for table in the database literature.) 


In [16]:
# Your code goes here. 

### BEGIN SOLUTION
dbListFields(connection, 'movies')
### END SOLUTION

[1] "id"         "title"      "orig_title" "start_year" "end_year"  
[6] "runtime"    "rating"     "nvotes"

### 2.1 Wrangling data with `dbplyr`

With `dbplyr`, we can work with a database table like it was loaded into memory (but it isn't!). 

To "read" a table from a database we can use the [dplyr::tbl](https://dplyr.tidyverse.org/reference/tbl.html) function. 

**Example**

Read the `movies` table from the `imdb` database.

In [17]:
(movies <- tbl(connection, 'movies'))

# Source:   table<movies> [?? x 8]
# Database: postgres [postgres@localhost:5432/imdb]
         id title         orig_title   start_year end_year runtime rating nvotes
      <int> <chr>         <chr>             <int>    <int>   <int>  <dbl>  <int>
 1 10035423 Kate & Leopo~ NA                 2001       NA     118    6.4  74982
 2 10042742 Mister 880    NA                 1950       NA      90    7.1   1171
 3 10041181 Black Hand    NA                 1950       NA      92    6.4    666
 4 10041387 Francis       NA                 1950       NA      91    6.4    979
 5 10041719 Orpheus       Orphée             1950       NA      95    8     9346
 6 10041931 Stromboli     Stromboli, ~       1950       NA     107    7.3   5239
 7 10042052 Woman in Hid~ NA                 1950       NA      92    6.9    553
 8 10042179 Abbott and C~ NA                 1950       NA      80    6.6   2573
 9 10042200 Annie Get Yo~ NA                 1950       NA     107    6.9   4050
10 10042206 Armored Ca

### 2.2 Lazy Evaluation and writting your own SQL queries

- Is there correlation between `runtime` and `rating`. What about `runtime` and `nvotes`? 

- What is the average `nvotes` for movies prior to 1990 and movies after 2010? Are people still interested in old movies? 

- Contrast the average rating for movies starring at least 3 people over 60 years. 

### 2.2 Plotting with data from a database

## 3. Writing to a database from R